https://www.booking.com

In [15]:
from selenium import webdriver
from fake_useragent import UserAgent
import time
from datetime import datetime, timedelta

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import logging
from urllib3.exceptions import NewConnectionError
from selenium.webdriver.support import expected_conditions as EC



In [16]:
def get_start_end_date():
    start_date = datetime.today()
    end_date = start_date + timedelta(days=60)
    
    return start_date, end_date

In [17]:
def next_link(driver, start_date):
    
    end_trip = start_date + timedelta(days=1)
    start_date_str = str(start_date).split(' ')
    end_trip_str = str(end_trip).split(' ')
    
    try:
        driver.get(f"https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIARXYAQHoAQGIAgGoAgO4Avjj3akGwAIB0gIkYzY5NTVhYzItYjQwYS00YWJlLWIyMWItOTc0OTAxNmVmYmNj2AIF4AIB&sid=ddc37917e9355cbdcad54b3c452b053a&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&checkin="+start_date_str[0]+"&checkout="+end_trip_str[0]+"&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure")
                      
    except NewConnectionError:
        logging.error(f"Erreur de connexion pour la date "+start_date+" et la date "+end_trip)
        try:
            driver.get(f"https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIARXYAQHoAQGIAgGoAgO4Avjj3akGwAIB0gIkYzY5NTVhYzItYjQwYS00YWJlLWIyMWItOTc0OTAxNmVmYmNj2AIF4AIB&sid=ddc37917e9355cbdcad54b3c452b053a&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&checkin="+start_date_str[0]+"&checkout="+end_trip_str[0]+"&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure")

        except Exception as e:
            logging.error(f"impossible de scrapper cette date : "+start_date+" et "+end_trip)
            driver.quit()
            return
    time.sleep(2)
    
    return end_trip

In [18]:
def get_url_list(driver):
    link_elements = driver.find_elements(By.CLASS_NAME, 'a78ca197d0')
    url_list = []
    
    for link_element in link_elements:
        url = link_element.get_attribute("href")
        if url is not None:
            url_list.append(url)
    print(url_list)
    return url_list

In [19]:
def get_data(driver):
    try:
        nom_chambre = driver.find_element(By.CSS_SELECTOR, '.d2fee87262').get_attribute('textContent')
        print(nom_chambre)
    except Exception as e:
        logging.error("we y a pas de nom?")
    try :
        prix_initiale = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)').get_attribute('textContent')
        print(prix_initiale)
    except Exception as e:
        logging.error("no reduction")
    try :
        taux_reduction = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)').get_attribute('textContent')
        print(taux_reduction) 
    except Exception as e:
        logging.error("no reduction")    
    try :
        prix_apres_reduction = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)').get_attribute('textContent')
        print(prix_apres_reduction)
    except Exception as e:
        logging.error("comment ca y a pas de prix?")
    time.sleep(2)

In [20]:
def go_to_url(url_list, driver):
    for url in url_list:
        try:
            driver.get(url)
        except Exception as e:
            logging.error("impossible de se connecter")
            logging.error("reessaie...")
            try:
                driver.get(url)
            except Exception as e:
                logging.error("pas de connexion frero")
                continue
        print(url)
        # get all necessary data from the pages 
        get_data(driver)

In [21]:
ua = UserAgent()
user_agent = ua.random

options = webdriver.FirefoxOptions()
options.add_argument(f'user-agent={user_agent}')
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)

start_date, end_date = get_start_end_date()

#récupère le reservations d'hotels sur une durée de 1 jour
while start_date < end_date:
    
    print("new date")
    
    #go to the next link
    end_trip = next_link(driver, start_date)
    # get a list of url
    url_list = get_url_list(driver)
    #go to the matching url 
    go_to_url(url_list, driver)  
    
    
    start_date = end_trip

driver.quit()

new date
['https://www.booking.com/hotel/fr/paris-j-39-adore-amp-spa.fr.html?label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIARXYAQHoAQGIAgGoAgO4Avjj3akGwAIB0gIkYzY5NTVhYzItYjQwYS00YWJlLWIyMWItOTc0OTAxNmVmYmNj2AIF4AIB&sid=ddc37917e9355cbdcad54b3c452b053a&aid=304142&ucfs=1&arphpl=1&checkin=2023-10-30&checkout=2023-10-31&dest_id=-1456928&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=2f5592c649dc03cd&srepoch=1698699153&all_sr_blocks=713048608_298165938_0_2_0&highlighted_blocks=713048608_298165938_0_2_0&matching_block_id=713048608_298165938_0_2_0&sr_pri_blocks=713048608_298165938_0_2_0__57563&from_sustainable_property_sr=1&from=searchresults#hotelTmpl', 'https://www.booking.com/hotel/fr/regina-paris.fr.html?label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIARXYAQHoAQGIAgGoAgO4Avjj3akGwAIB0gIkYzY5NTVhYzItYjQwYS00YWJlLWIyMWItOTc0OTAxNmVmYmNj2AIF4AIB&sid=ddc37917e9355cbdcad54b3c452b053a&aid=304142&ucfs=1&arphpl=

ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?


chic appartement centre paris


ERROR:root:no reduction
ERROR:root:no reduction



€ 208

Appartement luxe Avenue Foch /Arc de Triomphe ,champs Elysées air conditionne 4P


...

ERROR:root:no reduction
ERROR:root:no reduction



€ 693



ERROR:root:no reduction
ERROR:root:no reduction


Cosy apt Arc de Triomphe

€ 185

WIT Hotel

€ 143

-28 %

€ 103



ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?
ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?
ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?
ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?
ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?
ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?
ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?
